# import

In [1]:
import math
from pyproj import Transformer
import pandas as pd
from shapely.geometry import Polygon, Point
from ultralytics import YOLO
import pandas as pd
import os
import requests
from PIL import Image
from io import BytesIO

# CSV 파일 업로드 -> 픽셀 폴리곤 만들기

In [2]:
# UTM (GRS80, EPSG:5186) to WGS84 (EPSG:4326) 변환 함수
def transform_coords(utm_coords, from_epsg=5186, to_epsg=4326):
    utm_coords = list(utm_coords.exterior.coords)
    transformer = Transformer.from_crs(from_epsg, to_epsg, always_xy=True)
    latlon_coords = [transformer.transform(x, y) for x, y in utm_coords]
    return latlon_coords

# WGS84 좌표를 이미지 픽셀 좌표로 변환하는 함수
# zoom, pixel_distance, tile_size는 고정해놓았습니다. (나중에 변동 필요할거 같으면 그 때 바꿉시다)
def geo_to_pixel(lat, lon, center_lat, center_lon, zoom=18, pixel_distance=0.229, tile_size=512):
    # 위도 및 경도 거리 차이 계산
    delta_lat = lat - center_lat
    delta_lon = lon - center_lon

    # 위도 및 경도 거리 (미터) 변환
    d_lat = delta_lat * 111320  # 위도 1도당 약 111.32 km
    d_lon = delta_lon * (111320 * math.cos(math.radians(center_lat)))  # 경도 1도당 거리

    # 거리 차이를 픽셀 차이로 변환
    delta_y = d_lat / pixel_distance
    delta_x = d_lon / pixel_distance

    # 중심 기준 픽셀 좌표 계산 (이미지 중심은 tile_size / 2)
    center_pixel = tile_size / 2
    pixel_x = center_pixel + delta_x
    pixel_y = center_pixel - delta_y

    return pixel_x, pixel_y

def function(center_lat, center_lon, coords):
    # UTM 좌표를 WGS84 좌표로 변환
    latlon_coords = transform_coords(coords)

    # WGS84 좌표를 픽셀 좌표로 변환
    pixel_coords = [geo_to_pixel(lat, lon, center_lat, center_lon) for lon, lat in latlon_coords]

    # 폴리곤으로 변환
    polygon = Polygon(pixel_coords)

    return polygon

# 문자열을 Polygon 객체로 변환하는 함수 정의
def string_to_polygon(polygon_str):
    coordinates = polygon_str.replace('POLYGON ((', '').replace('))', '').replace(')', '').replace('(', '').split(', ')
    coordinates = [tuple(map(float, coord.split())) for coord in coordinates]
    return Polygon(coordinates)


# 여기에 csv파일 넣기
df = pd.read_csv('data.csv')

# csv파일로 저장하니까 polygon 형태가 str로 되어 있어서 (str -> 폴리곤) 형태를 바꿔 주어야 합니다.
# apply 메서드를 사용하여 데이터프레임의 열을 변환
df['geometry'] = df['geometry'].apply(string_to_polygon)

# 실제 폴리곤 데이터를 픽셀의 폴리곤 데이터로 변경
df['pixel_polygon'] = df.apply(lambda row: function(row['위도'], row['경도'], row['geometry']), axis=1)

# 이미지 파일 만들기

In [3]:
# 네이버 API 키
client_id = '11wep1dhly'  # 클라이언트 ID
client_secret = 'X9arTpnTbZKBwsr4xgJghBiA4oiAIXsjVX8clWP5'  # 클라이언트 Secret

def get_satellite_image(x, y):
    url = 'https://naveropenapi.apigw.ntruss.com/map-static/v2/raster'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'center': f'{x},{y}',
        'level': 18,
        'w': 512,
        'h': 512,
        'maptype': 'satellite_base',
        'scale': 1,
    }
    response = requests.get(url, headers=headers, params=params)
    image_bytes = BytesIO(response.content)
    satellite_image = Image.open(image_bytes)
    return satellite_image

# 이미지 파일로 모델 돌려서 모델 결과 df 만들기

In [8]:
def make_result_df(image):
    # 저장된 모델 파일 경로
    saved_model_path = 'best.pt'

    # 저장한 모델 불러오기
    loaded_model = YOLO(saved_model_path, task='detect')

    # 예측
    predictions = loaded_model.predict(source=image,
                            line_width=2)

    data_rows = []

    for i in range(len(predictions)):
        boxes = predictions[i].boxes

        num_elements = boxes.cls.size(0)

        for j in range(num_elements):
            x = boxes.xywh[j][0].item()
            y = boxes.xywh[j][1].item()
            conf = boxes.data[j][4].item()
            class_id = boxes.data[j][5].item()

            # 파일 이름 추출
            file_name, _ = os.path.splitext(os.path.basename(predictions[i].path))

            # 데이터 행 추가
            data_rows.append({'imgName': file_name, 'x': x, 'y': y, 'conf': conf, 'class': class_id})
        # data_rows.append({'x': x, 'y': y, 'conf': conf, 'class': class_id})

    df2 = pd.DataFrame(data_rows)

    return df2

# 폴리곤 안에 좌표 있는지 확인하기 (농지 위에 건물 있는지 확인)

In [9]:
def point_in_polygon(x, y, polygon):
    point = Point(x, y)
    return polygon.contains(point)

# 불법건출물 탐지 파이프 라인

In [10]:
def function(x, y, polygon):
    image_data = get_satellite_image(x, y)
    model_result_df = make_result_df(image_data)
    model_result_df['inside_polygon'] = None
    for i, row in model_result_df.iterrows():
        if ((row['class'] == 0) or (row['class'] == '0')) and (row['conf'] > 0.5):
            check = point_in_polygon(row['x'], row['x'], polygon)
            model_result_df.at[i,'inside_polygon'] = check  
    # model_result_df['inside_polygon'] = model_result_df.apply(lambda row: point_in_polygon(row['x'], row['y'], polygon), axis=1)
    value_counts = model_result_df['inside_polygon'].value_counts()
    count = value_counts.get(True, 0)
    return count

In [11]:
df['cnt_on_land'] = None

for i, row in df.iterrows():
    cnt = function(row['경도'], row['위도'], row['pixel_polygon'])
    df.at[i,'cnt_on_land'] = cnt

df


0: 640x640 7 buildings, 2 greenhouses, 156.6ms
Speed: 3.9ms preprocess, 156.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 9 buildings, 4 greenhouses, 173.6ms
Speed: 3.6ms preprocess, 173.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 greenhouses, 154.6ms
Speed: 3.5ms preprocess, 154.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 18 buildings, 2 greenhouses, 148.5ms
Speed: 3.5ms preprocess, 148.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 153.1ms
Speed: 3.3ms preprocess, 153.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 building, 148.6ms
Speed: 3.2ms preprocess, 148.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 buildings, 151.3ms
Speed: 3.6ms preprocess, 151.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 building, 153.9ms


,Unnamed: 0,고유번호,법정동코드,지목,토지면적,geometry,주소,공시지가,위도,경도,pixel_polygon,cnt_on_land
0,0,4418025023107940004,4418025023,전,77.0,POLYGON ((165444.01484912797 406853.3855316849...,충청남도 보령시 웅천읍 수부리 794-4,23900,36.259031,126.615412,"POLYGON ((283.1381833258266 265.8357357421483,...",0
1,1,4418035023103150004,4418035023,전,274.0,POLYGON ((169768.07070799381 422733.9706069558...,충청남도 보령시 청라면 내현리 315-4,13100,36.402165,126.662920,POLYGON ((279.74637565445914 206.8255637400483...,0
2,2,4423034033100430010,4423034033,답,1329.0,"POLYGON ((214323.08718311513 406380.175415892,...",충청남도 논산시 상월면 숙진리 43-10,18300,36.255295,127.159186,POLYGON ((328.8347493834407 273.93842562183823...,0
3,3,4480025321101860002,4480025321,전,1515.0,"POLYGON ((166063.55512473104 435161.417428017,...",충청남도 홍성군 광천읍 신진리 186-2,60500,36.514015,126.621439,POLYGON ((139.09154329495428 199.0821991174362...,3
4,4,4471031024100860001,4471031024,답,1637.0,POLYGON ((240341.91291475706 392757.7516145939...,충청남도 금산군 금성면 상가리 86-1,23600,36.131586,127.447908,POLYGON ((360.87271705102796 171.8455947942549...,0
5,5,4421025025105950000,4421025025,전,684.0,"POLYGON ((145529.3799732161 486055.4300342526,...",충청남도 서산시 대산읍 기은리 595,43800,36.971541,126.388386,POLYGON ((214.8325867322223 142.61970313482374...,0
6,6,4418025029100390001,4418025029,답,1302.0,POLYGON ((159152.65727765753 403655.0188282688...,충청남도 보령시 웅천읍 독산리 39-1,13300,36.229749,126.545737,POLYGON ((219.48653499575377 162.0502910841169...,1
7,7,4477041026105740003,4477041026,답,2618.0,POLYGON ((159094.91869583057 395154.0967911129...,충청남도 서천군 서면 신합리 574-3,13800,36.153056,126.545467,POLYGON ((247.21793935674833 122.1627622221121...,1
8,8,4418035022103430000,4418035022,답,344.0,"POLYGON ((171846.8539751024 422186.3139263629,...",충청남도 보령시 청라면 소양리 343,12200,36.397209,126.686245,POLYGON ((227.86336926345075 166.1462728231375...,0
9,9,4421010600108550006,4421010600,답,433.0,POLYGON ((153770.39775885077 463234.7736656772...,충청남도 서산시 수석동 855-6,191200,36.766555,126.482363,POLYGON ((198.41012129054337 240.8913345033638...,0
